In [3]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [4]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 05:16:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-11-30 05:16:51 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [21]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
Game = spark.read.csv(SparkFiles.get('amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz'), sep='\t', header=True)
Game.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [22]:
# Get the number of rows in the DataFrame.
Game.count()

4057147

# Transform the Data

## Create the "review_id_table".

In [12]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = Game.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date', 'yyyy-MM-dd').alias('review_date')])
review_id_df.show()

Row(review_id='R3ARRMDEGED8RD', customer_id='22873041', product_id='B00KJWQIIC', product_parent='335625766', review_date=datetime.date(2015, 8, 31))

## Create the "products" Table

In [10]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = Game.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0016P392O|iMicro SP-IM942 M...|
|B00PPRYEZA|CoulVue Holder an...|
|B00VZXRRGG|Hexagon Network -...|
|B007GF9N96|Computer USB-Powe...|
|B00LCK3TEG|           Fire HD 6|
|B00KMOG5FS|Fintie 10.1" Tabl...|
|B00CFL5CRI|Cable Matters 2-P...|
|B008NP2AUC|USB USB-2, USB-3 ...|
|B00OAJ5N6I|Samsung 850 EVO -...|
|B00H02XN2I|Techno Earth Rubb...|
|B00YSC6UW8|5 Mm Thick Office...|
|B00CRXZPC6|Lenovo Ideatab S6...|
|B009SKTZFY|Bosign Tabletpill...|
|B003NZSX7G|Netgear N300 WiFi...|
|B0089JIDLC|Kingston ValueRAM...|
|B001E45XT4|Plantronics GameC...|
|B002IY4V9C|Swiss Gear Men's ...|
|B000UVRU6G|SteelSeries QcK G...|
|B00X5X3LQY|EVGA Pro SLI Brid...|
|B00L49X8E6|ASUS X551 15.6-in...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = Game.groupBy('customer_id').agg({'customer_id': 'count'}).withColumnRenamed('count(customer_id','customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [15]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = Game.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3ARRMDEGED8RD|          5|            0|          0|   N|
| RQ28TSA020Y6J|          5|           24|         31|   N|
| RUXJRZCT6953M|          1|            2|          2|   N|
| R7EO0UO6BPB71|          1|            0|          0|   N|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|
|R31SR7REWNX7CF|          5|            0|          0|   N|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|
|R1QF6RS1PDLU18|          4|            1|          1|   N|
|R23AICGEDAJQL1|          1|            0|          0|   N|
|R2EY3N4K9W19UP|          5|            3|          4|   N|
| RC9AW4HKJ016M|          1|            0|          0|   N|
|R2ALWJE9N6ZBXD|          1|            0|          0|   N|
|R2G5FPA4OX37GV|          5|            1|          1|   N|
|R1IKTSEVXSIMOD|          5|            

# Load

In [19]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class"
config = {"user":"postgres", "password": "postgresql", "driver":"org.postgresql.Driver"}

In [23]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='reviews', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)